# Day 24 - Qwen QwQ 32B

In [1]:
import collections
import re

def simulate_gates(input_file):
    # Read the input file and parse initial wire values and gate connections
    wire_values = {}
    gates = {}
    wire_dependencies = {}
    with open(input_file, 'r') as f:
        lines = f.readlines()
        initial_values = [line.strip() for line in lines if ':' in line]
        connections = [line.strip() for line in lines if ':' not in line]

        # Parse the initial wire values
        for line in initial_values:
            wire, value = line.split(':')
            wire_values[wire.strip()] = int(value.strip())

        # Parse the gate connections
        for line in connections:
            match = re.match(r'(\w+) (AND|OR|XOR) (\w+) -> (\w+)', line)
            if match:
                output, op, a, b = match.groups()
                gates[output] = (op, a, b)
                wire_dependencies.setdefault(a, []).append(output)
                wire_dependencies.setdefault(b, []).append(output)

    # Initialize a queue with gates that have both input wires in wire_values
    queue = collections.deque([gate for gate in gates if wire_values[gates[gate][1]] is not None and wire_values[gates[gate][2]] is not None])

    # Function to compute output based on operation
    operations = {
        'AND': lambda a, b: int(a & b),
        'OR': lambda a, b: int(a | b),
        'XOR': lambda a, b: int(a ^ b)
    }

    while queue:
        gate = queue.popleft()
        a, b = wire_values[gates[gate][1]], wire_values[gates[gate][2]]
        if a is None or b is None:
            continue
        output = operations[gates[gate][0]](a, b)
        wire_values[gates[gate][0]] = output
        for dependent in wire_dependencies[gates[gate][0]]:
            if wire_values[gates[dependent][1]] is not None:
                queue.append(dependent)

    # Collect the values of all wires starting with 'z'
    z_wires = [wire for wire in wire_values if wire.startswith('z')]
    z_values = [wire_values[wire] for wire in z_wires]

    # Combine the values into a binary string and convert to decimal
    binary_string = ''.join(str(value) for value in z_values)
    decimal_value = int(binary_string, 2)

    return decimal_value

print(simulate_gates('input.txt'))

KeyError: 'rcw'

In [2]:
def main():
    import re
    from collections import deque, defaultdict

    # Read input from file
    with open("input.txt", "r") as file:
        lines = [line.strip() for line in file.readlines()]

    # Separate initial values and gate connections
    initial_values = {}
    gate_connections = []
    parsing_gates = False
    for line in lines:
        if not line:
            continue
        if not parsing_gates and re.match(r'\w+:\s\d', line):
            parts = line.split(':')
            wire = parts[0].strip()
            value = int(parts[1].strip())
            initial_values[wire] = value
        else:
            parsing_gates = True
            gate_connections.append(line)

    # Parse gate connections
    gates = []
    for connection in gate_connections:
        parts = connection.split(' -> ')
        output_wire = parts[1].strip()
        inputs_and_type = parts[0].strip()
        if 'AND' in inputs_and_type:
            gate_type = 'AND'
            inputs = inputs_and_type.split(' AND ')
        elif 'OR' in inputs_and_type:
            gate_type = 'OR'
            inputs = inputs_and_type.split(' OR ')
        elif 'XOR' in inputs_and_type:
            gate_type = 'XOR'
            inputs = inputs_and_type.split(' XOR ')
        else:
            raise ValueError(f"Unknown gate type: {inputs_and_type}")
        input_wires = [inp.strip() for inp in inputs]
        gates.append({'type': gate_type, 'inputs': input_wires, 'output': output_wire})

    # Initialize wire values
    wire_values = initial_values.copy()

    # Build dependency graph
    dependencies = defaultdict(list)
    input_requirements = defaultdict(int)
    for gate in gates:
        input_wires = gate['inputs']
        output_wire = gate['output']
        for wire in input_wires:
            if wire not in initial_values:
                input_requirements[gate] += 1
                dependencies[wire].append(gate)

    # Queue for gates that are ready to compute (both inputs are available)
    ready_gates = deque()
    for gate in gates:
        if input_requirements[gate] == 0:
            ready_gates.append(gate)

    # Simulate the circuit
    while ready_gates:
        gate = ready_gates.popleft()
        input_wires = gate['inputs']
        output_wire = gate['output']
        gate_type = gate['type']

        # Get input values
        try:
            value1 = wire_values[input_wires[0]]
            value2 = wire_values[input_wires[1]]
        except KeyError:
            # Input wire not yet set, skip this gate for now
            continue

        # Compute output based on gate type
        if gate_type == 'AND':
            output_value = value1 & value2
        elif gate_type == 'OR':
            output_value = value1 | value2
        elif gate_type == 'XOR':
            output_value = value1 ^ value2
        else:
            raise ValueError(f"Unknown gate type: {gate_type}")

        # Set the output wire
        wire_values[output_wire] = output_value

        # Update dependencies
        for dep_gate in dependencies[output_wire]:
            input_requirements[dep_gate] -= 1
            if input_requirements[dep_gate] == 0:
                ready_gates.append(dep_gate)

    # Collect 'z' wires and form the binary number
    z_wires = [wire for wire in wire_values if wire.startswith('z')]
    z_wires_sorted = sorted(z_wires, key=lambda x: int(x[1:]))
    binary_str = ''.join(str(wire_values[wire]) for wire in z_wires_sorted)
    decimal_output = int(binary_str, 2)

    print(decimal_output)

if __name__ == "__main__":
    main()

TypeError: unhashable type: 'dict'

In [3]:
from collections import deque, defaultdict

class Gate:
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
    
    def compute_output(self, wire_values):
        raise NotImplementedError("Subclasses should implement this method.")

class AndGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] and wire_values[self.inputs[1]]

class OrGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] or wire_values[self.inputs[1]]

class XorGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] != wire_values[self.inputs[1]]

def parse_input_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    initial_values = {}
    gate_connections = {}
    
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:  # Initial value assignment
            wire, value = parts
            initial_values[wire] = int(value)
        elif len(parts) == 5:  # Gate connection
            input1, operation, input2, arrow, output = parts
            gate_id = f"{input1}_{operation}_{input2}"
            if operation == "AND":
                gate = AndGate([input1, input2], [output])
            elif operation == "OR":
                gate = OrGate([input1, input2], [output])
            elif operation == "XOR":
                gate = XorGate([input1, input2], [output])
            else:
                raise ValueError(f"Unknown operation: {operation}")
            gate_connections[gate_id] = gate
        else:
            raise ValueError(f"Invalid line format: {line}")
    
    return initial_values, gate_connections

def main():
    initial_values, gate_connections = parse_input_file('input.txt')
    
    # Initialize data structures
    wire_values = initial_values.copy()
    dependencies = defaultdict(list)
    input_requirements = {}
    
    for gate in gate_connections.values():
        input_requirements[gate] = len(gate.inputs)
        for wire in gate.inputs:
            if wire not in initial_values:
                dependencies[wire].append(gate)
    
    # Queue for gates that are ready to compute (both inputs are available)
    ready_gates = deque()
    for gate in gate_connections.values():
        if input_requirements[gate] == 0:
            ready_gates.append(gate)
    
    while ready_gates:
        current_gate = ready_gates.popleft()
        # Compute output
        output_value = current_gate.compute_output(wire_values)
        for output_wire in current_gate.outputs:
            wire_values[output_wire] = output_value
            # Update dependencies
            for dependent_gate in dependencies.get(output_wire, []):
                input_requirements[dependent_gate] -= 1
                if input_requirements[dependent_gate] == 0:
                    ready_gates.append(dependent_gate)
    
    # Collect z-wires and form the binary number
    z_wires = sorted([wire for wire in wire_values if wire.startswith('z')], key=lambda x: int(x[1:]))
    binary_representation = ''.join(str(wire_values[wire]) for wire in z_wires)
    decimal_output = int(binary_representation, 2)
    print(decimal_output)

if __name__ == "__main__":
    main()

ValueError: Invalid line format: 


In [4]:
from collections import deque, defaultdict

class Gate:
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
    
    def compute_output(self, wire_values):
        raise NotImplementedError("Subclasses should implement this method.")

class AndGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] and wire_values[self.inputs[1]]

class OrGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] or wire_values[self.inputs[1]]

class XorGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] != wire_values[self.inputs[1]]

def parse_input_file(file_path):
    initial_values = {}
    gate_connections = {}
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('Set wire '):
                # Parse initial wire values
                wire_id, value = line[9:].split(' = ')
                initial_values[wire_id] = int(value)
            elif line.startswith('NOT '):
                # Parse not gate
                gate_id, input_wire = line[4:].split(' -> ')
                gate = {'type': 'NOT', 'input': input_wire, 'output': gate_id}
                gate_connections[gate_id] = gate
            elif line.startswith('AND '):
                # Parse and gate
                gate_id, input_wires = line[4:].split(' -> ')
                gate = {'type': 'AND', 'inputs': input_wires.split(', '), 'output': gate_id}
                gate_connections[gate_id] = gate
            elif line.startswith('OR '):
                # Parse or gate
                gate_id, input_wires = line[3:].split(' -> ')
                gate = {'type': 'OR', 'inputs': input_wires.split(', '), 'output': gate_id}
                gate_connections[gate_id] = gate
            else:
                raise ValueError(f"Invalid line format: {line}")
    return initial_values, gate_connections

def main():
    initial_values, gate_connections = parse_input_file('input.txt')
    
    # Initialize data structures
    wire_values = initial_values.copy()
    dependencies = defaultdict(list)
    input_requirements = {}
    
    for gate in gate_connections.values():
        input_requirements[gate] = len(gate.inputs)
        for wire in gate.inputs:
            if wire not in initial_values:
                dependencies[wire].append(gate)
    
    # Queue for gates that are ready to compute (both inputs are available)
    ready_gates = deque()
    for gate in gate_connections.values():
        if input_requirements[gate] == 0:
            ready_gates.append(gate)
    
    while ready_gates:
        current_gate = ready_gates.popleft()
        # Compute output
        output_value = current_gate.compute_output(wire_values)
        for output_wire in current_gate.outputs:
            wire_values[output_wire] = output_value
            # Update dependencies
            for dependent_gate in dependencies.get(output_wire, []):
                input_requirements[dependent_gate] -= 1
                if input_requirements[dependent_gate] == 0:
                    ready_gates.append(dependent_gate)
    
    # Collect z-wires and form the binary number
    z_wires = sorted([wire for wire in wire_values if wire.startswith('z')], key=lambda x: int(x[1:]))
    binary_representation = ''.join(str(wire_values[wire]) for wire in z_wires)
    decimal_output = int(binary_representation, 2)
    print(decimal_output)

if __name__ == "__main__":
    main()

ValueError: Invalid line format: x00: 1

In [5]:
from collections import deque, defaultdict

class Gate:
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
    
    def compute_output(self, wire_values):
        raise NotImplementedError("Subclasses should implement this method.")

class AndGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] and wire_values[self.inputs[1]]

class OrGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] or wire_values[self.inputs[1]]

class XorGate(Gate):
    def compute_output(self, wire_values):
        return wire_values[self.inputs[0]] != wire_values[self.inputs[1]]

def parse_input_file(file_path):
    initial_values = {}
    gate_connections = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        # Parse initial values
        for line in lines:
            line = line.strip()
            if line:
                if '->' in line:  # gate connection
                    continue
                wire_id, value = line.split(':')
                initial_values[wire_id.strip()] = int(value.strip())
        
        # Parse gate connections
        for line in lines:
            line = line.strip()
            if not line:  # Blank line, separator
                continue
            gate_connections[line.split(' -> ')[1]] = line.split(' -> ')[0]
    return initial_values, gate_connections

def main():
    initial_values, gate_connections = parse_input_file('input.txt')
    
    # Initialize data structures
    wire_values = initial_values.copy()
    dependencies = defaultdict(list)
    input_requirements = {}
    
    for gate in gate_connections.values():
        input_requirements[gate] = len(gate.inputs)
        for wire in gate.inputs:
            if wire not in initial_values:
                dependencies[wire].append(gate)
    
    # Queue for gates that are ready to compute (both inputs are available)
    ready_gates = deque()
    for gate in gate_connections.values():
        if input_requirements[gate] == 0:
            ready_gates.append(gate)
    
    while ready_gates:
        current_gate = ready_gates.popleft()
        # Compute output
        output_value = current_gate.compute_output(wire_values)
        for output_wire in current_gate.outputs:
            wire_values[output_wire] = output_value
            # Update dependencies
            for dependent_gate in dependencies.get(output_wire, []):
                input_requirements[dependent_gate] -= 1
                if input_requirements[dependent_gate] == 0:
                    ready_gates.append(dependent_gate)
    
    # Collect z-wires and form the binary number
    z_wires = sorted([wire for wire in wire_values if wire.startswith('z')], key=lambda x: int(x[1:]))
    binary_representation = ''.join(str(wire_values[wire]) for wire in z_wires)
    decimal_output = int(binary_representation, 2)
    print(decimal_output)

if __name__ == "__main__":
    main()

IndexError: list index out of range

## Part 2